## NeuroDet CNN 2 CLass

In [1]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import PIL
import PIL.Image
import matplotlib.pyplot as plt
import pathlib
import math


# Disabling GPU for the moment because of the lack of the memory
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1" 
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
train_path = 'BrainMRI_2Class/Training'
test_path = 'BrainMRI_2Class/Testing'

#train_glioma_dir = pathlib.Path(train_path + categories_path['glioma_tumor'])
train_dir = pathlib.Path(train_path)
test_dir = pathlib.Path(test_path)

# Defining the parameters of the dataset
batch_size = 32 #was 128 reduce to 16 to fit the data on the gpu ram
img_height = 256 # was 128
img_width = 256 # was 128

# Loading the train dataset using keras.utils.image_dataset_from_directory
# To use this method, please ensure you have tf.nigthly installed 
train_ds = tf.keras.utils.image_dataset_from_directory(train_dir, seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size, color_mode = 'grayscale')

test_ds = tf.keras.utils.image_dataset_from_directory(test_dir, seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size, color_mode = 'grayscale')

print(train_ds.class_names)
print(test_ds.class_names)

Found 2870 files belonging to 2 classes.
Found 394 files belonging to 2 classes.
['no_tumor', 'tumor']
['no_tumor', 'tumor']


In [3]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))
print(len(train_ds))

90


In [4]:
# Got 80% acc at first but then removing the relu on the first dense layer helped push acc to 85%
# 30 epochs seems to be instable, I moved to 35
# changed previous class from 4 to 10. At 4, rerunning the model was producing high variance 73~85, even with 50epoch.
class CNNTumorBin(tf.keras.Model):
    def __init__(self, num_classes):
        super(CNNTumorBin, self).__init__()
        
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        
         #previous kernel filter number  that produced .75 was 10. 16 filter is not good
        self.norm1_1 = tf.keras.layers.BatchNormalization()
        self.norm1_2 = tf.keras.layers.BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding ="same")
        self.conv2 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding ="same")
        self.max1 = tf.keras.layers.MaxPooling2D(padding ="same")
        
        
        self.norm2_1 = tf.keras.layers.BatchNormalization()
        self.norm2_2 = tf.keras.layers.BatchNormalization()
        self.conv3 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding ="same")
        self.conv4 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding ="same")
        self.max2 = tf.keras.layers.MaxPooling2D(padding ="same")
        
        self.norm3 = tf.keras.layers.BatchNormalization()
        #self.conv5 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding ="same")
        self.max3 = tf.keras.layers.MaxPooling2D(padding ="same")
        
        self.flat = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(80)
        self.dense2 = tf.keras.layers.Dense(10, activation = 'relu')
        self.out = tf.keras.layers.Dense(num_classes, activation = 'softmax')
        
    def call(self, batch_input, is_training=False):
        c1 = self.conv1(batch_input)
        c1 = tf.nn.relu(self.norm1_1(c1, training=is_training))
        c2 = self.conv2(c1)
        c2 = tf.nn.relu(self.norm1_2(c2, training=is_training))
        m1 = self.max1(c2)
        
        #print("m_1 shape")
        #print(m1.shape)
        
        c3 = self.conv3(m1)
        c3 = tf.nn.relu(self.norm2_1(c3, training=is_training))
        c3 = self.conv3(c3)
        c3 = tf.nn.relu(self.norm2_2(c3, training=is_training))
        m2 = self.max2(c3)
        
        #print("m_2 shape")
        #print(m2.shape)
        
        c4 = self.conv4(m2)
        c4 = tf.nn.relu(self.norm3(c4, training=is_training))
        m3 = self.max3(c4)
        
        #print("m_3 shape")
        #print(m3.shape)
        
        # residual add
        #resNet_add = batch_input + m3
        
        flat = self.flat(m3)
        dense1 = self.dense1(flat)
        desen2 = self.dense2(dense1)
        
        return self.out(desen2)
    
    def loss(self, probas, labels):
        return tf.reduce_mean(tf.keras.metrics.sparse_categorical_crossentropy(labels, probas))
    
    def train(self, train_dataset):
        list_loss = []
        for images, labels in  train_dataset:
            with tf.GradientTape() as tape:
                probas = self.call(images, is_training = True)
                loss = self.loss(probas, labels)
                acc = self.accuracy(probas, labels)
                #print("Loss for the batch {}, accuracy {}".format(loss, acc))
                list_loss.append(loss)
                
            gradients = tape.gradient(loss, self.trainable_variables)
            self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        print("avg Loss for the epoch {}".format(np.average(np.array(list_loss))))
        

    def test(self, test_dataset):
        # maybe return all information to also output roc plots?
        accs = []
        for images, labels in test_dataset:
            probas = self.call(images)
            acc = self.accuracy(probas, labels)
            accs.append(acc)
            
        return accs
        
    
    def accuracy(self, probas, labels):
        correct_predictions = tf.equal(tf.argmax(probas, 1), tf.cast(labels, tf.int64))
        return tf.reduce_mean(tf.cast(correct_predictions, tf.float32))

In [5]:
cnn_tumorbin = CNNTumorBin(2)

for i in range(50):
    print("epoch "+str(i+1))
    cnn_tumorbin.train(train_ds)

epoch 1
avg Loss for the epoch 0.31639277935028076
epoch 2
avg Loss for the epoch 0.201790913939476
epoch 3
avg Loss for the epoch 0.17838150262832642
epoch 4
avg Loss for the epoch 0.1526583880186081
epoch 5
avg Loss for the epoch 0.12563054263591766
epoch 6
avg Loss for the epoch 0.12136561423540115
epoch 7
avg Loss for the epoch 0.14048883318901062
epoch 8
avg Loss for the epoch 0.11786632239818573
epoch 9
avg Loss for the epoch 0.11222001910209656
epoch 10
avg Loss for the epoch 0.10187883675098419
epoch 11
avg Loss for the epoch 0.12866449356079102
epoch 12
avg Loss for the epoch 0.12818990647792816
epoch 13
avg Loss for the epoch 0.09967473149299622
epoch 14
avg Loss for the epoch 0.09778390824794769
epoch 15
avg Loss for the epoch 0.08730661123991013
epoch 16
avg Loss for the epoch 0.09635381400585175
epoch 17
avg Loss for the epoch 0.09952192008495331
epoch 18
avg Loss for the epoch 0.07684851437807083
epoch 19
avg Loss for the epoch 0.07966846227645874
epoch 20
avg Loss for th

In [9]:
print(tf.reduce_mean(cnn_tumorbin.test(test_ds)))

tf.Tensor(0.82067305, shape=(), dtype=float32)
